## 7.1 不同种类的变量存储

变量和对象存储在内存的不同区域，具体取决于它们在C++程序中的声明方式。
这会影响数据缓存的效率（请参见第89页）。
如果数据在内存中随处随机分散，则数据缓存效率很差。
因此了解变量的存储方式非常重要。
这个存储的相关原则对于简单变量，数组和对象是相同的。

### 栈上的存储

在函数内部声明的变量和对象存储在堆栈中，下面部分描述的情况除外。

堆栈是以先入后出方式组织的一部分内存。
它用于存储函数返回地址（即函数被调用的地方），函数参数，局部变量，以及用于保存函数返回前必须恢复的寄存器。
每次调用函数时，都会因此而在堆栈中分配所需的空间。
该内存空间在函数返回时被释放。
然后再调用下一个函数时，它可以为新函数的参数重用相同的空间。

该堆栈是用于存储数据的最有效的存储空间，因为同一范围的内存地址被一次又一次地重复使用。
如果没有大数组，那么几乎可以肯定的是，这部分内存被镜像在一级数据缓存中，访问速度非常快。

我们可以从中学到的教训是，所有变量和对象都应该在使用它们的函数内声明。

通过在{}括号内声明变量，可以使变量的范围更小。
但是，直到函数返回，大多数编译器不会释放变量使用的内存，即使在退出声明变量的{}括号作用域时它本可以释放内存。
如果变量存储在一个寄存器中（见下文），那么在函数返回之前它就可能被释放。

### 全局或静态存储

在函数之外声明的变量称为全局变量。他们可以从任何函数访问。
全局变量存储在内存的静态部分。
静态内存也用于使用`static`关键字声明的变量，浮点常量，字符串常量，数组初始化列表，`switch`语句跳转表和虚函数表。

静态数据区通常分为三类：
- 程序永远不会修改的常量，
- 程序可修改的初始化变量，
- 未初始化的变量，可由程序修改。

静态数据的优缺点：
- 优点是可以在程序启动之前将其初始化为所需的值。
- 缺点是在整个程序执行过程中占用了内存空间，即使该变量仅用于程序的一小部分。这使数据缓存效率降低。

如果可以避免，不要将变量设置为全局变量。
不同线程之间的通信可能需要全局变量，但这是它们不可避免的唯一情况。
如果几个不同的函数需要访问，并且想要避免将变量作为函数参数传输的开销，则创建一个变量全局可能很有用。
但是一个可能更好的解决方案是，把需要访问该变量的函数设计成类成员函数，同时把该共享变量也成为同一个类的数据成员。
喜欢哪种解决方案是编程风格的问题。


通常更倾向于让使查找表静态化。例如：

```cpp
// Example 7.1
float SomeFunction (int x) {
    static float list[] = {1.1, 0.3, -2.0, 4.4, 2.5};
    return list[x];
}
```

这里使用静态的优点是在调用函数时不需要执行列表初始化。当程序加载到内存时，这些值就已经存在那里。
如果从上面的例子中删除了`static`这个词，那么每次调用该函数时，所有五个值都必须被动态地放入列表中。
这是通过将整个列表从静态内存复制到堆栈内存来完成的。
**
在大多数情况下，将常量数据从静态存储器复制到堆栈是时间的浪费。但在特殊情况下，如果数据需要在循环中多次，这种情况可能是最优的，在这种情况下，几乎整个一级缓存被利用起来，用于你想在堆栈中保持在一起的数组。
**


字符串常量和浮点常量存储在静态内存中（以优化的代码的形式）。例如：

```cpp
// Example 7.2
a = b * 3.5;
c = d + 3.5;
```

这里，常数3.5将被存储在静态存储器中。
大多数编译器会认识到这两个常量是相同的，所以只需要存储一个常量。
整个程序中所有相同的常量将被连接在一起，以尽量减少用于常量的缓存空间。

整型常量通常包含在指令代码中。**你可以假设整数常量没有缓存问题。**


### 寄存器存储

有限数量的变量可以存储在寄存器中，而不是主存储器中。
寄存器是用于临时存储的CPU内部的一小块内存。
存储在寄存器中的变量非常快速地被访问。
所有优化编译器都会自动选择函数中最常用的变量来存储寄存器。
只要其用途（活动范围）不重叠，相同的寄存器可用于多个变量。

寄存器的数量非常有限。在32位操作系统中有大约六个整数寄存器可用于通用目的，在64位系统中有十四个整数寄存器。

浮点变量使用不同类型的寄存器。
在32位操作系统中有8个浮点寄存器，在64位操作系统中有16个浮点寄存器。
除非启用了SSE2指令集（或更高版本），否则一些编译器难以在32位模式下使用浮点寄存器变量。


### Volatile

`volatile`关键字指定：一个变量可以被别的线程改变。
编译器可能会假定变量始终具有之前在代码中赋予的值，基于这种假设来进行某些优化。该关键字可以防止此类优化的发生。
例如：

```cpp
// Example 7.3. 解释 volatile
volatile int seconds; // 每秒会被其它的线程+1
void DelayFiveSeconds() {
    seconds = 0;
    while (seconds < 5) {
        // 5秒前，什么也不做
    }
}
```

在这个例子中，`DelayFiveSeconds`函数将等待，直到变量`seconds`由另一个线程增加到5。
如果seconds没有被声明为`volatile`，那么优化编译器会假定while循环中的`seconds`保持为零，因为循环中没有代码可以更改该值。
循环将是`while（0 <5）{}`，这将是一个无限循环。

关键字`volatile`的作用是确保变量存储在内存中而不是寄存器中，并防止对变量的所有优化。
这可能在测试情况下很有用，以避免某些表达式被优化掉。

请注意，`volatile`并不意味着原子性。
它不会阻止两个线程同时尝试写入变量。
上面示例中的代码在这样的情况下可能会失败：其它线程增加`seconds`的同时尝，此线程将`seconds`设置为零。
一个更安全的实现将只读取秒的值，并等待该值改变五次。

### 线程本地存储

大多数编译器可以使用关键字`__declspec`或`__declspec(thread)`来创建静态和全局变量的线程本地存储。
这些变量对于每个线程都有一个实例。
**线程本地存储效率低下，因为它是通过存储在线程环境块中的指针进行访问的。**
如果可能的话，应该避免线程局部存储，并用栈中的存储替换（参见上文，第26页）
存储在栈上的变量总是属于创建它们的线程。

### Far

具有分段内存的系统（如DOS和16位Windows）允许通过使用关键词`far`（数组也可以很大）将变量存储在远端数据段中。
`far`存储，`far`指针和`far`程过程效率低下。
如果某个程序的某个段的数据太多，则建议使用允许更大的段（32位或64位系统）的不同操作系统。


### 动态内存分配

动态内存分配是通过运算符`new`和`delete`或`malloc`和`free`函数完成的。这些操作员和功能消耗大量时间。
一部分内存被保留出来，被称为堆，用于动态分配。
当随机顺序分配和释放不同大小的对象时，堆容易变得碎片化。
堆管理器可以花费大量时间清理不再使用的空间，并搜索空闲空间。这被称为垃圾收集。
以一定顺序分配的一组对象，不一定按顺序存储在内存中。
当堆已经变得分散时，它们可能散布在不同的地方。这使得数据缓存效率低下。

动态内存分配也会使代码更加复杂且容易出错。
程序必须保持指向所有分配的对象的指针，并跟踪它们何时不再使用。
在所有可能的程序流程情况下，所有分配的对象也都被释放是非常重要的。
没能这样做，会造成一种常见的错误，称为内存泄漏。
更糟糕的一种错误是在释放对象后访问对象。
程序逻辑可能需要额外的开销来防止这种错误。

有关使用动态内存分配的优点和缺点，请参见第92页。

一些编程语言（如Java）为所有对象使用动态内存分配。这当然是低效的。

### 类中声明的变量

在类中声明的变量按它们出现在类声明中的顺序存储。
存储类型是在声明类的对象时确定的。
类，结构或联合的对象可以使用上面提到的任何存储方法。
除了最简单的情况外，对象不能存储在寄存器中，但其数据成员可以复制到寄存器中。

具有`static`修饰符的类成员变量将存储在静态内存中，并且将只有一个实例。
同一个类中的非静态成员将与类的每个实例一起存储。

将变量存储在类或结构中，这是确保在程序的相同部分中使用的变量存储也彼此邻近的好方法。
有关使用类的优点和缺点，请参阅第52页。